### Self-Refine Prompt Engineering

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)

Based on this [research](https://selfrefine.info/) - self-refine is a way to incorporate the LLM itself into the loop when trying to obtain information that is most relevant to your original query.

Instead of having a human attempt to produce better prompts - we allow the LLM itself to act as judge and arbiter of its own outputs.

In [ ]:
!pip install openai -q

### OpenAI API Key

In [ ]:
import os
import openai

# set the OPENAI_API_KEY environment variable
openai.api_key = ""

### Continue Prompting

In [ ]:
from IPython.display import display, Markdown

def get_response(messages: str) -> str:
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )["choices"][0]["message"]

def wrap_prompt(message: str, role: str) -> dict:
    return {"role": role, "content": message}

def m_print(message: str) -> str:
    display(Markdown(message["content"]))

In [ ]:
prompt = wrap_prompt("Can you write me a function in Python that calculates the Nth Fibonacci number?", "user")
system = wrap_prompt("You are a Python Programmer.", "system")

m_print(get_response([system, prompt]))

Sure, here's a simple function in Python to calculate the Nth Fibonacci number using recursion:

```python
def fibonacci(n):
    if n <= 0:
        raise ValueError("The input for the Fibonacci sequence must be a positive integer.")
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

# Test the function.
n = 10
print(f"The {n}th Fibonacci number is: {fibonacci(n)}")
```

Note that this recursive approach is not the most efficient way to calculate large Fibonacci numbers due to its exponential time complexity. For larger numbers, you might want to use a more efficient approach like memoization or iteration.

In [ ]:
system_prompt = wrap_prompt("You are a Python Programmer. Please keep your answers short and concise - but use markdown.", "system")
prompt = wrap_prompt("Can you write me a function in Python that calculates the Nth Fibonacci number?", "user")
refine_prompt = wrap_prompt("Can you optimize this code to have a lower time complexity?", "user")

conversation = [system_prompt, prompt]

initial_output = get_response(conversation)

m_print(initial_output)

conversation += [initial_output, refine_prompt]

refined_output = get_response(conversation)

print("\n\n")

m_print(refined_output)

Here is the function that calculates the Nth Fibonacci number using recursion:

```python
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)
```

However, using recursion for large N may result in poor performance. To avoid this, you can use dynamic programming or memoization. Below is an example using memoization:

```python
def fibonacci_memo(n, memo=None):
    if memo is None:
        memo = {}
    if n in memo:
        return memo[n]
    if n <= 1:
        result = n
    else:
        result = fibonacci_memo(n - 1, memo) + fibonacci_memo(n - 2, memo)
    memo[n] = result
    return result
```

Yes, we can optimize the code using an iterative method which has a lower time complexity (O(n)). Here's the function:

```python
def fibonacci_iter(n):
    if n <= 1:
        return n

    fib_n_minus_2, fib_n_minus_1 = 0, 1
    for _ in range(2, n+1):  # Iterate from 2 to n
        fib_n = fib_n_minus_2 + fib_n_minus_1
        fib_n_minus_2, fib_n_minus_1 = fib_n_minus_1, fib_n  # Update values for the next iteration
    
    return fib_n_minus_1
```

This function calculates the Nth Fibonacci number iteratively with a time complexity of O(n).

In [ ]:
def check_refinement(conversation_history: list, refinement_question: str):
    conversation_history += [refinement_question]
    return get_response(conversation_history)["content"] == "Yes"

refined_prompt = wrap_prompt("Is this as optimized as you can get it? Please only answer Yes or No. No other answers will be accepted.", "system")

In [ ]:
conversation += [refined_output, refine_prompt]

refined_output = get_response(conversation)

print("\n\n")

m_print(refined_output)

refined = check_refinement(conversation, refined_prompt)

if refined:
    print("This is as optimized as we can go.")
else:
    conversation += [refined_output, refine_prompt]

    refined_output = get_response(conversation)

    print("\n\n")

    m_print(refined_output)

Yes, we can optimize the code further using matrix exponentiation, which reduces the time complexity to O(log(n)). Here's the function:

```python
def matrix_mult(a, b):
    result = [[0, 0],
              [0, 0]]
    for i in range(2):
        for j in range(2):
            for k in range(2):
                result[i][j] += a[i][k] * b[k][j]
    return result


def matrix_pow(matrix, power):
    if power == 1:
        return matrix
    elif power % 2 == 0:
        half_pow = matrix_pow(matrix, power // 2)
        return matrix_mult(half_pow, half_pow)
    else:
        return matrix_mult(matrix, matrix_pow(matrix, power - 1))


def fibonacci_matrix(n):
    if n <= 1:
        return n

    matrix = [[1, 1],
              [1, 0]]
    result_matrix = matrix_pow(matrix, n-1)

    # First element of the result matrix's first row is the Fibonacci number
    return result_matrix[0][0]
```

This function calculates the Nth Fibonacci number using matrix exponentiation with a time complexity of O(log(n)).

This is as optimized as we can go.


In [ ]:
def refine(conversation_history: list, refinement_prompt: dict, fully_refined_prompt: dict, n_iterations: int = 10):
    initial_output = get_response(conversation_history)

    m_print(initial_output)

    for i in range(n_iterations):
        conversation_history += [initial_output, refinement_prompt]

        refined_output = get_response(conversation_history)

        print("\n\n")

        m_print(refined_output)

        refined = check_refinement(conversation_history, fully_refined_prompt)

        if refined:
            print("This is as optimized as we can go.")
            break

        initial_output = refined_output

In [ ]:
conversation = [
    wrap_prompt("You are a Python Programmer. Please keep your answers short and concise - but use markdown.", "system"),
    wrap_prompt("Given two sorted arrays nums1 and nums2 of size m and n respectively, return the median of the two sorted arrays.", "user")
]

refinement_prompt = wrap_prompt("Can you optimize this code to have a lower time complexity?", "user")

fully_refined_prompt = wrap_prompt("Is this as optimized as you can get it? Please only answer Yes or No. No other answers will be accepted.", "system")

refine(conversation, refinement_prompt, fully_refined_prompt)

You can achieve this by merging the two sorted arrays and then finding the median element(s) depending on the length of the merged array. Here's a Python function to do that:

```python
def findMedianSortedArrays(nums1, nums2):
    nums = sorted(nums1 + nums2)
    length = len(nums)

    if length % 2 == 0:
        return (nums[length//2 - 1] + nums[length//2]) / 2
    else:
        return nums[length//2]
```

You can use this function to find the median of two sorted arrays like this:

```python
nums1 = [1, 3]
nums2 = [2]
median = findMedianSortedArrays(nums1, nums2)
print(median)  # Output: 2
```

Sure! We can use a binary search algorithm to optimize the code and reduce the time complexity to O(log(min(m, n))). Here's the optimized version:

```python
def findMedianSortedArrays(nums1, nums2):
    if len(nums1) > len(nums2):
        nums1, nums2 = nums2, nums1

    m, n = len(nums1), len(nums2)
    imin, imax, half_len = 0, m, (m + n + 1) // 2
    
    while imin <= imax:
        i = (imin + imax) // 2
        j = half_len - i
        
        if i < m and nums2[j - 1] > nums1[i]:
            imin = i + 1
        elif i > 0 and nums1[i - 1] > nums2[j]:
            imax = i - 1
        else:
            if i == 0: max_of_left = nums2[j - 1]
            elif j == 0: max_of_left = nums1[i - 1]
            else: max_of_left = max(nums1[i - 1], nums2[j - 1])

            if (m + n) % 2 == 1:
                return max_of_left

            if i == m: min_of_right = nums2[j]
            elif j == n: min_of_right = nums1[i]
            else: min_of_right = min(nums1[i], nums2[j])

            return (max_of_left + min_of_right) / 2.0
```

You can use this optimized function to find the median of two sorted arrays like this:

```python
nums1 = [1, 3]
nums2 = [2]
median = findMedianSortedArrays(nums1, nums2)
print(median)  # Output: 2.0
```

No

This is as optimized as we can go.


In [ ]:
conversation = [
    wrap_prompt("You are a linguistic expert. Given the following sentence - can you reverse its sentiment?", "system"),
    wrap_prompt("I really love going to the movies and eating delicious pizza.", "user")
]

refinement_prompt = wrap_prompt("In your professional linguistic opinion, is this sentiment exactly reversed? If not, why?", "user")

fully_refined_prompt = wrap_prompt("Is this as reversed as the sentiment can be? Please answer only Yes or No. No other answers will be accepted.", "system")

refine(conversation, refinement_prompt, fully_refined_prompt)

I really hate going to the movies and eating terrible pizza.

In my professional linguistic opinion, the sentiment is effectively reversed. The original sentence expresses positive feelings about going to the movies and eating delicious pizza. In the revised sentence, the sentiment is flipped to convey negative feelings towards the same activities by using the words "hate" instead of "love" and "terrible" instead of "delicious." However, if we are looking for an exact reversal, we might want to ensure that each part of the sentence is equally negated. Here's an alternative reversed sentiment:

I really despise staying home and eating tasteless pizza.

This is as optimized as we can go.
